# 01. Исследование данных криптовалют

Этот ноутбук предназначен для первичного анализа данных из PostgreSQL

In [ ]:
# Импорты
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Настройка визуализации
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Импорт модулей проекта
from data.data_loader import CryptoDataLoader
from utils.logger import setup_logging
import yaml

# Загрузка конфигурации
with open('../config/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

setup_logging()
print("Библиотеки загружены успешно!")

## 1. Загрузка данных

In [ ]:
# Инициализация загрузчика данных
data_loader = CryptoDataLoader(config)

# Загрузка данных
print("Загрузка данных из PostgreSQL...")
df = data_loader.load_raw_data(limit=100000)  # Ограничиваем для быстрой загрузки

print(f"\nЗагружено {len(df)} записей")
print(f"Период: {df['datetime'].min()} - {df['datetime'].max()}")
print(f"Количество символов: {df['symbol'].nunique()}")

## 2. Общая информация о данных

In [ ]:
# Структура данных
print("Структура DataFrame:")
df.info()

print("\nПервые 5 записей:")
df.head()

In [ ]:
# Базовая статистика
print("Описательная статистика:")
df.describe()

In [ ]:
# Проверка пропущенных значений
print("Пропущенные значения:")
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_values,
    'Percentage': missing_percent
})
missing_df[missing_df['Missing Count'] > 0]

## 3. Анализ по символам

In [ ]:
# Топ символов по количеству записей
symbol_counts = df['symbol'].value_counts().head(20)

plt.figure(figsize=(12, 6))
symbol_counts.plot(kind='bar')
plt.title('Топ-20 символов по количеству записей')
plt.xlabel('Символ')
plt.ylabel('Количество записей')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Анализ объемов торгов по символам
volume_by_symbol = df.groupby('symbol')['volume'].agg(['sum', 'mean', 'std']).sort_values('sum', ascending=False).head(20)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Общий объем
volume_by_symbol['sum'].plot(kind='bar', ax=ax1)
ax1.set_title('Общий объем торгов по символам')
ax1.set_xlabel('Символ')
ax1.set_ylabel('Объем')
ax1.tick_params(axis='x', rotation=45)

# Средний объем
volume_by_symbol['mean'].plot(kind='bar', ax=ax2, color='orange')
ax2.set_title('Средний объем торгов по символам')
ax2.set_xlabel('Символ')
ax2.set_ylabel('Средний объем')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 4. Временной анализ

In [ ]:
# Выбираем популярный символ для детального анализа
selected_symbol = symbol_counts.index[0]
symbol_data = df[df['symbol'] == selected_symbol].copy()
symbol_data = symbol_data.sort_values('datetime')

print(f"Анализ для символа: {selected_symbol}")
print(f"Количество записей: {len(symbol_data)}")
print(f"Период: {symbol_data['datetime'].min()} - {symbol_data['datetime'].max()}")

In [ ]:
# График цены
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Цена закрытия
axes[0].plot(symbol_data['datetime'], symbol_data['close'], label='Close', linewidth=1)
axes[0].fill_between(symbol_data['datetime'], symbol_data['low'], symbol_data['high'], alpha=0.3, label='High-Low range')
axes[0].set_ylabel('Цена')
axes[0].set_title(f'{selected_symbol} - Динамика цены')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Объем
axes[1].bar(symbol_data['datetime'], symbol_data['volume'], width=0.01, alpha=0.7)
axes[1].set_ylabel('Объем')
axes[1].set_title('Объем торгов')
axes[1].grid(True, alpha=0.3)

# Волатильность (High-Low)
symbol_data['volatility'] = (symbol_data['high'] - symbol_data['low']) / symbol_data['close'] * 100
axes[2].plot(symbol_data['datetime'], symbol_data['volatility'], color='red', linewidth=1)
axes[2].set_ylabel('Волатильность (%)')
axes[2].set_xlabel('Дата')
axes[2].set_title('Внутридневная волатильность')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Анализ доходности

In [ ]:
# Расчет доходностей
symbol_data['returns'] = symbol_data['close'].pct_change()
symbol_data['log_returns'] = np.log(symbol_data['close'] / symbol_data['close'].shift(1))

# Статистика доходностей
print("Статистика доходностей:")
print(f"Средняя доходность: {symbol_data['returns'].mean():.4%}")
print(f"Стандартное отклонение: {symbol_data['returns'].std():.4%}")
print(f"Sharpe Ratio (годовой): {np.sqrt(252*96) * symbol_data['returns'].mean() / symbol_data['returns'].std():.2f}")
print(f"Максимальная доходность: {symbol_data['returns'].max():.4%}")
print(f"Минимальная доходность: {symbol_data['returns'].min():.4%}")

In [ ]:
# Распределение доходностей
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Гистограмма доходностей
axes[0, 0].hist(symbol_data['returns'].dropna(), bins=100, alpha=0.7, edgecolor='black')
axes[0, 0].axvline(symbol_data['returns'].mean(), color='red', linestyle='--', label='Среднее')
axes[0, 0].set_xlabel('Доходность')
axes[0, 0].set_ylabel('Частота')
axes[0, 0].set_title('Распределение доходностей')
axes[0, 0].legend()

# QQ-plot
from scipy import stats
stats.probplot(symbol_data['returns'].dropna(), dist="norm", plot=axes[0, 1])
axes[0, 1].set_title('QQ-plot доходностей')

# Кумулятивная доходность
cumulative_returns = (1 + symbol_data['returns']).cumprod()
axes[1, 0].plot(symbol_data['datetime'], cumulative_returns)
axes[1, 0].set_xlabel('Дата')
axes[1, 0].set_ylabel('Кумулятивная доходность')
axes[1, 0].set_title('Кумулятивная доходность')
axes[1, 0].grid(True, alpha=0.3)

# Rolling volatility
rolling_vol = symbol_data['returns'].rolling(window=96*7).std() * np.sqrt(252*96)  # Недельная волатильность
axes[1, 1].plot(symbol_data['datetime'], rolling_vol)
axes[1, 1].set_xlabel('Дата')
axes[1, 1].set_ylabel('Волатильность (годовая)')
axes[1, 1].set_title('Скользящая волатильность (7 дней)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Корреляционный анализ

In [ ]:
# Подготовка данных для корреляционного анализа
top_symbols = symbol_counts.head(10).index.tolist()
pivot_data = df[df['symbol'].isin(top_symbols)].pivot(index='datetime', columns='symbol', values='close')
returns_matrix = pivot_data.pct_change().dropna()

# Корреляционная матрица
correlation_matrix = returns_matrix.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Корреляционная матрица доходностей (Топ-10 символов)')
plt.tight_layout()
plt.show()

## 7. Анализ внутридневных паттернов

In [ ]:
# Добавляем временные признаки
df['hour'] = pd.to_datetime(df['datetime']).dt.hour
df['day_of_week'] = pd.to_datetime(df['datetime']).dt.dayofweek
df['day_name'] = pd.to_datetime(df['datetime']).dt.day_name()

# Средний объем по часам
hourly_volume = df.groupby('hour')['volume'].mean()

plt.figure(figsize=(12, 6))
hourly_volume.plot(kind='bar')
plt.title('Средний объем торгов по часам (UTC)')
plt.xlabel('Час')
plt.ylabel('Средний объем')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Волатильность по дням недели
df['price_range'] = (df['high'] - df['low']) / df['close'] * 100
daily_volatility = df.groupby('day_name')['price_range'].mean()
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_volatility = daily_volatility.reindex(day_order)

plt.figure(figsize=(10, 6))
daily_volatility.plot(kind='bar', color='coral')
plt.title('Средняя внутридневная волатильность по дням недели')
plt.xlabel('День недели')
plt.ylabel('Волатильность (%)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 8. Экспорт результатов

In [ ]:
# Сохранение основной статистики
summary_stats = {
    'total_records': len(df),
    'date_range': f"{df['datetime'].min()} - {df['datetime'].max()}",
    'unique_symbols': df['symbol'].nunique(),
    'top_symbols': symbol_counts.head(10).to_dict(),
    'missing_values': missing_df[missing_df['Missing Count'] > 0].to_dict(),
    'selected_symbol_stats': {
        'symbol': selected_symbol,
        'mean_return': float(symbol_data['returns'].mean()),
        'std_return': float(symbol_data['returns'].std()),
        'sharpe_ratio': float(np.sqrt(252*96) * symbol_data['returns'].mean() / symbol_data['returns'].std())
    }
}

# Сохранение в JSON
import json
with open('data_exploration_summary.json', 'w') as f:
    json.dump(summary_stats, f, indent=4, default=str)

print("Результаты сохранены в data_exploration_summary.json")
print("\nКраткая сводка:")
for key, value in summary_stats.items():
    if isinstance(value, dict):
        print(f"{key}:")
        for k, v in list(value.items())[:3]:
            print(f"  {k}: {v}")
    else:
        print(f"{key}: {value}")